# Key, ID ve Header Tanımlamaları

In [1]:
import requests
# Gerekli olan bilgileri Unity Cloud sistminde bulunan projenin ayarlar kısmından alabilirsiniz.
projectID = "..."
environmentID = "..."
organizationID = "..."
apiKEY = "..."

headers = {
    'Authorization': f'Basic {apiKEY}',
}

# Kullanılacak olan URL'ler

In [2]:
# https://services.docs.unity.com/cloud-save-admin/v1/

list_players_with_items = f"https://services.api.unity.com/cloud-save/v1/data/projects/{projectID}/environments/{environmentID}/players"

get_player_items = f"https://services.api.unity.com/cloud-save/v1/data/projects/{projectID}/environments/{environmentID}/players/{{playerId}}/items"

### Kullanıcı ID'lerini ve dosya boyutlarını çekmek için sorgu

In [3]:
response = requests.get(list_players_with_items, headers=headers)

json_data = response.json()

user_ids = []
access_classes = []
num_keys = []
total_sizes = []

for result in json_data['results']:
    user_ids.append(result['id']) 
    access_classes.append(result['accessClasses']) 
    for access_class in result['accessClasses'].values():
        num_keys.append(access_class['numKeys']) 
        total_sizes.append(access_class['totalSize']) 

print("Kullanıcı ID'leri:", user_ids)
print("Toplam Boyutlar:", total_sizes)

Kullanıcı ID'leri: ['0ee51JRG9HGNePVvS5r5guVklHJ0', '2fa7thDdlqOzACztstMNs2PVYbBW', 'AOOglvIxRd4kulEDXsG01ELQJIhk', 'COoGeUE7nRZvrPPwrcoAs98ig18n', 'EiPZXDf4oAETzUkhR2g5qunWlv7H', 'EjNYPMGrsh1jcXdlWkP7zMQqg4wf', 'MyF4eRTMdGKs0Q6mCxRFLewW9CBV', 'uRO6QyeKpFtDe983gxYkJ6qdNsmE', 'xRSu3OfvKypaqaPuAtu53Yn6Ijhk']
Toplam Boyutlar: [8, 15, 15, 12, 4, 8, 12, 15, 4]


### Bir kullanıcı ID'si için veri sorgulama

In [4]:
user_id = "EjNYPMGrsh1jcXdlWkP7zMQqg4wf" # Örnek bir kullanıcı ID'si

get_player_items_url = f"https://services.api.unity.com/cloud-save/v1/data/projects/{projectID}/environments/{environmentID}/players/{user_id}/items"

headers = {'Authorization': f'Basic {apiKEY}'}

response = requests.get(get_player_items_url, headers=headers)
items_data = response.json()

print(items_data)

{'results': [{'key': 'green_door', 'value': True, 'writeLock': 'b2b995a08f35ba7a4b2b6e606962373a', 'modified': {'date': '2024-02-26T19:18:26Z'}, 'created': {'date': '2024-02-26T17:12:13Z'}, 'storedSize': 4}, {'key': 'red_door', 'value': True, 'writeLock': 'cf32d11fc5b07ff42c50ffdc839433e0', 'modified': {'date': '2024-02-26T19:18:33Z'}, 'created': {'date': '2024-02-26T17:12:13Z'}, 'storedSize': 4}], 'links': {'next': None}, 'sizeLimit': 5242880, 'totalSize': 8}


### Tüm kullanıcıları için ID ve veri eşleştirmesini sağlayan sorgu

In [5]:
response = requests.get(list_players_with_items, headers=headers)
players_data = response.json()

user_ids = [player['id'] for player in players_data['results']]

player_items = {}
for user_id in user_ids:
    get_player_items_url = f"https://services.api.unity.com/cloud-save/v1/data/projects/{projectID}/environments/{environmentID}/players/{user_id}/items"
    response = requests.get(get_player_items_url, headers=headers)
    items_data = response.json()

    items_list = [{
        'key': item['key'],
        'value': item['value'],
        'created': item['created']['date'],
        'modified': item['modified']['date']
    } for item in items_data['results']]

    player_items[user_id] = items_list

for user_id, items in player_items.items():
    print(f"User ID: {user_id}, Items: {items}")

User ID: 0ee51JRG9HGNePVvS5r5guVklHJ0, Items: [{'key': 'blue_door', 'value': True, 'created': '2024-02-26T21:14:40Z', 'modified': '2024-02-26T21:14:42Z'}, {'key': 'red_door', 'value': True, 'created': '2024-02-26T21:14:40Z', 'modified': '2024-02-26T21:14:40Z'}]
User ID: 2fa7thDdlqOzACztstMNs2PVYbBW, Items: [{'key': 'blue_door', 'value': False, 'created': '2024-02-26T16:05:52Z', 'modified': '2024-02-26T16:16:48Z'}, {'key': 'green_door', 'value': False, 'created': '2024-02-26T16:05:52Z', 'modified': '2024-02-26T16:16:48Z'}, {'key': 'red_door', 'value': False, 'created': '2024-02-26T16:05:52Z', 'modified': '2024-02-26T16:16:48Z'}]
User ID: AOOglvIxRd4kulEDXsG01ELQJIhk, Items: [{'key': 'blue_door', 'value': False, 'created': '2024-02-26T15:46:53Z', 'modified': '2024-02-26T15:46:53Z'}, {'key': 'green_door', 'value': False, 'created': '2024-02-26T15:46:53Z', 'modified': '2024-02-26T15:46:53Z'}, {'key': 'red_door', 'value': False, 'created': '2024-02-26T15:46:53Z', 'modified': '2024-02-26T15: